# Final Project for DTSA 5304 - Intro to Data Visualizations at the University of Colorado Masters of Data Science Degree

## This workbook will serve as an analysis of all Pokemon stats through Generation 6, which was released in 2013. 

## This analysis will be done in Python3, using the Altair library for the Visualizations

### **Step 1: load the dataset into a data frame**

In [7]:
# Import our data processing library (note: you may have to install this!)
import pandas as pd
import altair as alt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff 
import plotly.graph_objs as go 
from plotly.subplots import make_subplots
import requests
from io import StringIO
from IPython.display import HTML
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

url = 'https://raw.githubusercontent.com/rdadmun/Data_Science_Degree/main/DTSA_5304/Pokemon_Stats.csv'
response = requests.get(url)
pkmn_data = pd.read_csv(StringIO(response.text))
pkmn_data.head()

/var/folders/d8/j2s681kd64q1hp65blg721040000gn/T/ipykernel_17754/4208532635.py:5: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/var/folders/d8/j2s681kd64q1hp65blg721040000gn/T/ipykernel_17754/4208532635.py:6: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


### Defining colors for various Pokemon Types

In [3]:
#Defining Colors
colors = {
    "Bug": "#ABC206",
    "Dark": "#4A3A2F",
    "Dragon": "#5F21F6",
    "Electric": "#F3E313",
    "Fairy": "#EC83B7",
    "Fighting": "#C6231C",
    "Fire": "#F57C22",
    "Flying": "#A58CEB",
    "Ghost": "#684E8A",
    "Grass": "#76C945",
    "Ground": "#D6B55E",
    "Ice": "#9CE0DD",
    "Normal": "#B3B288",
    "Poison": "#B23BAF",
    "Psychic": "#FF467E",
    "Rock": "#B6A136",
    "Steel": "#C5C5D3",
    "Water": "#4C7CE2",
}

## Radar Charts
Next, we want to create a basic radar chart for statistic distributions for single Pokemon.

We have loaded multiple other libraries in order for this graph to work, as Altair doees not currently support radar charts. However, this is a standard inclusion in Pokemon games, so I wanted to include it. 

In [5]:
def polar_pokemon_stats(pkmn, pkmn_name, colors):
    pkmn_data = pkmn[pkmn.Name == pkmn_name]
    if pkmn_data.empty:
        raise ValueError(f"No data found for Pokemon: {pkmn_name}")
        
    obj = go.Scatterpolar(
        r=[
            pkmn_data['HP'].values[0],
            pkmn_data['Attack'].values[0],
            pkmn_data['Defense'].values[0],
            pkmn_data['Sp. Atk'].values[0],
            pkmn_data['Sp. Def'].values[0],
            pkmn_data['Speed'].values[0],
            pkmn_data['HP'].values[0]
        ],
        theta=[
            'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'HP'
        ],
        fill='toself',
        marker=dict(
            color=colors.get(pkmn_data['Type 1'].values[0], 'gray')
        ),
        name=pkmn_data['Name'].values[0]
    )
    return obj

def plot_single_pokemon(name):
    layout = go.Layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 250]
            )
        ),
        showlegend=False,
        title="Stats of {}".format(name)
    )

    pokemon_figure = go.Figure(data=[polar_pokemon_stats(pkmn_data, name, colors)], layout=layout)
    iplot(pokemon_figure, filename='Single Pokemon')

plot_single_pokemon('Mewtwo')

This is pretty good, but we want to ensure we include a dropdown menu for users

In [9]:
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='pokemon-dropdown',
        options=[{'label': pokemon, 'value': pokemon} for pokemon in pkmn_data['Name']],
        value='Bulbasaur',  # Default selected Pokémon
        style={'width': '50%'}
    ),
    dcc.Graph(id='pokemon-stats-plot')
])

# Define callback to update the plot based on dropdown selection
@app.callback(
    Output('pokemon-stats-plot', 'figure'),
    [Input('pokemon-dropdown', 'value')]
)
def update_plot(selected_pokemon):
    pkmn_data_selected = pkmn_data[pkmn_data['Name'] == selected_pokemon]
    
    if pkmn_data_selected.empty:
        raise ValueError(f"No data found for Pokemon: {selected_pokemon}")

    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{'type': 'polar'}]]
    )

    fig.add_trace(
        go.Scatterpolar(
            r=pkmn_data_selected.iloc[0, [5, 6, 7, 8, 9, 10, 5]],
            theta=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'HP'],
            fill='toself',
            marker=dict(color=colors.get(pkmn_data_selected['Type 1'].values[0], 'gray')),
            name=pkmn_data_selected['Name'].values[0]
        )
    )

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 250]
            )
        ),
        showlegend=False,
        title=f"Stats of {selected_pokemon}"
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)